In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('../dataset/mushrooms.csv')

In [4]:
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
data['class'].unique()

array(['p', 'e'], dtype=object)

In [6]:
ind = list(data.index)
np.random.shuffle(ind)

train_len = int(data.shape[0]*0.75)
train_ind = ind[:train_len]
training_data = data.iloc[train_ind,:]
#training_data.head()

test_ind = ind[train_len:]
testing_data = data.iloc[test_ind,:]
#testing_data.head()

print('Training_data size -> {}'.format(training_data.shape))
print('Testing_data size -> {}'.format(testing_data.shape))

assert data.shape[0] ==  len(train_ind)+ len(test_ind), 'Not equal distribution'

Training_data size -> (6093, 23)
Testing_data size -> (2031, 23)


\begin{equation}
P(Cap-shape \cap Cap-surface \cap ......\cap habitat | Class = p)\\[10pt]
P(Cap-shape \cap Cap-surface \cap ......\cap habitat | Class = e)
\end{equation}

In [7]:
class NB:
    def __init__(self, target, dataframe):
        self.df = dataframe
        # Target/Category Column
        self.c_n = target
        # Column Names
        self.cols = list(self.df.columns)
        self.cols.remove(self.c_n)
        
        self.store = {}
        self.likelihood_for_all_()
        
    def likelihood_cal(self, x, y, z):
        """ 
        x -> Column Name (String)
        y -> Column Value (String)
        z -> Class value (String)
        c_n -> Class Name (Target)
        
        Returns -> P(x = y | c_n = z)
        """
        df = self.df
        
        if x not in self.cols:
            raise KeyError("Feature(column) not present in the Training Dataset")
        
        res =  len(df[(df[x] == y) & (df[self.c_n] == z)]) /len(df[df[self.c_n] == z])
        
        if res == 0.0:
            return 1/(len(df[df[self.c_n] == z]) + len(df[x].unique()))
        
        return res
    
    def likelihood_for_all_(self):     
        df = self.df
        
        dict1 = {}
        for x in self.cols:
            dict2 = {}
            for y in df[x].unique():
                dict3 = {}
                for z in df[self.c_n].unique():
                    #print('P({}="{}"|{}="{}") = {}'.format(x,y,self.c_n,z,self.likelihood_cal(x, y, z)))
                    dict3[z] = self.likelihood_cal(x, y, z)
                dict2[y] = dict3
            dict1[x] = dict2
        
        self.store = dict1
    
    def likelihood_expr(self, class_val, expr):
        val = 1  
        
        for k,v in expr:
            try:
                store_val = self.store[k][v][class_val]
            except:
                store_val = self.likelihood_cal(k,v,class_val)
                
            val *= store_val
                                         
        return val
    
    def prior(self, class_val):
        df = self.df
        return len(df[df[self.c_n] == class_val])/df.shape[0]
    
    def predict(self, X):
        df = self.df
        
        if type(X) == pd.core.series.Series:
            values_list = [list(X.items())]
            
        elif type(X) == pd.core.frame.DataFrame:
            values_list = [list(y.items()) for x,y in X.iterrows()]
            
        else:
            raise TypeError('{} is not supported type'.format(type(X)))
            
        
        predictions_list = []
        for values in values_list:
            likelihood_priors = {}
            for class_val in df[self.c_n].unique():
                likelihood_priors[class_val] = self.prior(class_val)*self.likelihood_expr(class_val,values)
            #print(likelihood_priors)
            
            normalizing_prob = np.sum([x for x in likelihood_priors.values()])
            probabilities = [(y/normalizing_prob,x) for x,y in likelihood_priors.items()]
            #print(probabilities)
            
            if len(probabilities) == 2:
                # For 2 Class Predictions
                max_prob = max(probabilities)[1]
                predictions_list.append(max_prob)
            
            else:
                # For Mulit Class Predictions
                exp_1 = [np.exp(x) for x,y in probabilities]
                exp_2 = np.sum(exp_1)
                softmax = exp_1/exp_2
                #print(softmax)
                class_names = [y for x,y in probabilities]
                softmax_values = [(x,y) for x,y in zip(softmax,class_names)]
                #print(softmax_values)
                max_prob = max(softmax_values)[1]
                predictions_list.append(max_prob)
        
        
        return predictions_list
    
    def accuracy_score(self, X, Y):
        assert len(X) == len(Y), 'Given values are not equal in size'
        
        total_matching_values = [x == y for x,y in zip(X,Y)]
        return (np.sum(total_matching_values)/len(total_matching_values))*100
    
    def calculate_confusion_matrix(self, X, Y):
        df = self.df
        
        unique_class_values = df[self.c_n].unique()
        decimal_class_values = list(range(len(unique_class_values)))
        numerical = {x:y for x,y in zip(unique_class_values, decimal_class_values)}
        
        x = [numerical[x] for x in X]
        y = [numerical[y] for y in Y]
        
        
        n = len(decimal_class_values)
        confusion_matrix = np.zeros((n,n))
        
        for i,j in zip(x,y):
            if i == j:
                confusion_matrix[i][i] += 1
            elif i != j:
                confusion_matrix[i][j] += 1
        
        return confusion_matrix
            
    
    def precision_score(self, X, Y):
        assert len(X) == len(Y), 'Given values are not equal in size'
        
        confusion_matrix = self.calculate_confusion_matrix(X,Y)
        tp = confusion_matrix[0][0]
        fp = confusion_matrix[1][0]
        
        return tp / (tp+fp)
    
    def recall_score(self, X, Y):
        assert len(X) == len(Y), 'Given values are not equal in size'
        
        confusion_matrix = self.calculate_confusion_matrix(X,Y)
        tp = confusion_matrix[0][0]
        fn = confusion_matrix[0][1]
        
        return tp / (tp+fn)
                
                


In [8]:
genx = NB(target='class',dataframe=training_data)

In [9]:
testing_data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
2066,e,x,y,e,t,n,f,c,b,p,...,s,p,p,p,w,o,p,n,v,d
7532,e,x,s,w,f,n,f,w,b,p,...,k,w,w,p,w,t,p,w,n,g
680,e,x,s,w,t,l,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
6787,p,x,y,e,f,s,f,c,n,b,...,s,w,w,p,w,o,e,w,v,l
4383,p,x,y,y,f,f,f,c,b,h,...,k,n,b,p,w,o,l,h,v,p


In [10]:
y_test = list(testing_data.iloc[:,0])
y_pred = genx.predict(testing_data.iloc[:,1:])
#print(y_test)
#print(y_pred)

In [11]:
print('Accuracy Score -> {} %'.format(round(genx.accuracy_score(y_test,y_pred),3)))
print('Precison Score -> {}'.format(round(genx.precision_score(y_test,y_pred),3)))
print('Recall Score -> {}'.format(round(genx.recall_score(y_test,y_pred),3)))

Accuracy Score -> 95.372 %
Precison Score -> 0.92
Recall Score -> 0.995
